In [2]:
!pip install tensorflow fastapi uvicorn onnx onnxruntime pillow matplotlib numpy ultralytics
!pip install fastapi uvicorn onnx onnxruntime pillow matplotlib numpy torch torchvision python-multipart
!pip install albumentations

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 896.9/896.9 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 3.6 MB/s eta 0:00:00


Mount Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import shutil
import albumentations as A
from albumentations.core.composition import OneOf
from albumentations.augmentations import transforms as T
from PIL import Image
import numpy as np
from tqdm import tqdm

In [5]:
Fish_cat_dir ="/content/drive/MyDrive/Fish_cat"
Hyena_dir ="/content/drive/MyDrive/Hyena"

In [6]:


def augment_data(base_dir, augment_per_image=5):
    """
    Creates an augmented version of the base directory, saving both original and augmented images.
    If the augmented directory exists, it is removed and recreated.

    Args:
        base_dir (str): Path to the root directory containing category subfolders with images.
        augment_per_image (int): Number of augmentations to create per image.
    """
    # Create aug_folder path by replacing the last folder name with 'aug_' prefix
    augment_target_dir = base_dir.split("/")
    augment_target_dir[-1] = 'aug_' + augment_target_dir[-1]
    aug_folder = "/".join(augment_target_dir)

    print(f"Augmented folder: {aug_folder}")

    # If aug_folder exists, remove it
    if os.path.exists(aug_folder):
        print(f"Augmented folder already exists. Removing it...")
        shutil.rmtree(aug_folder)
    os.makedirs(aug_folder)

    # Define augmentation pipeline
    augmentations = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Rotate(limit=30, p=0.7),
        A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.7),
        A.Blur(blur_limit=5, p=0.3),
        A.CLAHE(p=0.3),
        A.ColorJitter(p=0.3),
        A.CoarseDropout(max_holes=8, max_height=8, max_width=8, p=0.5),
        A.RandomShadow(p=0.3),
    ])

    count = 1  # Unique numerical naming
    for category in os.listdir(base_dir):
        category_path = os.path.join(base_dir, category)
        if os.path.isdir(category_path):
            # Create corresponding category folder in aug_folder
            aug_category_path = os.path.join(aug_folder, category)
            os.makedirs(aug_category_path, exist_ok=True)

            print(f"Processing category: {category}")
            for img_file in tqdm(os.listdir(category_path)):
                img_path = os.path.join(category_path, img_file)
                try:
                    # Open image
                    image = np.array(Image.open(img_path).convert("RGB"))

                    # Save original image
                    Image.fromarray(image).save(os.path.join(aug_category_path, f"{count}.jpg"))
                    count += 1

                    # Generate and save augmented images
                    for _ in range(augment_per_image):
                        augmented = augmentations(image=image)['image']
                        Image.fromarray(augmented).save(os.path.join(aug_category_path, f"{count}.jpg"))
                        count += 1

                except Exception as e:
                    print(f"Failed to process {img_path}: {e}")

    print(f"Augmented dataset created in '{aug_folder}' with {count - 1} images.")

# # Example usage
# augment_data(Fish_cat_dir, augment_per_image=5)

# Model YAML creation

In [7]:
import os
import shutil
from sklearn.model_selection import train_test_split
import yaml

def prepare_yolo_dataset(main_dir, output_dir, val_ratio=0.2, yaml_path="dataset.yaml"):
    """
    Converts a dataset into YOLOv8-compatible structure and generates a YAML config.

    Args:
        main_dir (str): Path to the original dataset directory.
        output_dir (str): Path to the output directory.
        val_ratio (float): Ratio of validation data.
        yaml_path (str): Path to save the YAML configuration file.
    """
    train_dir = os.path.join(output_dir, "train")
    val_dir = os.path.join(output_dir, "val")
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)

    class_names = []

    # Process each subdirectory (class folder)
    for class_name in os.listdir(main_dir):
        class_path = os.path.join(main_dir, class_name)
        if os.path.isdir(class_path):
            class_names.append(class_name)

            # List all images in the class folder
            images = [os.path.join(class_path, img) for img in os.listdir(class_path) if img.endswith(('.png', '.jpg', '.jpeg'))]

            # Split into train and val
            train_imgs, val_imgs = train_test_split(images, test_size=val_ratio)

            # Create class subdirectories in train/val folders
            os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
            os.makedirs(os.path.join(val_dir, class_name), exist_ok=True)

            # Copy images to train/val folders
            for img in train_imgs:
                shutil.copy(img, os.path.join(train_dir, class_name, os.path.basename(img)))
            for img in val_imgs:
                shutil.copy(img, os.path.join(val_dir, class_name, os.path.basename(img)))

    # Create YAML file
    yaml_content = {
        "train": os.path.abspath(train_dir),
        "val": os.path.abspath(val_dir),
        "nc": len(class_names),
        "names": class_names
    }
    with open(yaml_path, "w") as yaml_file:
        yaml.dump(yaml_content, yaml_file, default_flow_style=False)

    print(f"Dataset prepared at {output_dir}")
    print(f"YAML file created at {yaml_path}")

# # Example Usage
# main_directory = Fish_cat_dir
# output_directory = "/content/drive/MyDrive/Fish_cat_yolo"
# yaml_file_path = output_directory + "/dataset.yaml"  # YAML config file path

# prepare_yolo_dataset(main_directory, output_directory, val_ratio=0.2, yaml_path=yaml_file_path)

# Training Pipeline

In [11]:
from ultralytics import YOLO
def train_pipeline(base_dir):
    target_dir = base_dir.split("/")
    target_dir[-1] = 'aug_' + target_dir[-1]
    aug_folder = "/".join(target_dir)

    target_dir[-1] = 'yolo_' + target_dir[-1][4:]
    yolo_folder = "/".join(target_dir)

    # Define yaml_file_path here
    yaml_file_path = os.path.join(yolo_folder, "dataset.yaml")

    # Data pre-processing
    augment_data(base_dir, augment_per_image=5)
    prepare_yolo_dataset(aug_folder, yolo_folder, val_ratio=0.2, yaml_path=yaml_file_path)

    # training
    model = YOLO('yolov8n-cls.yaml')  # 'n' is for nano, choose 's', 'm', 'l' or 'x' for larger models

    # Train the model
    model.train(
        data= yolo_folder , # Path to the dataset
        epochs=50,    # Number of training epochs
        imgsz=224,    # Image size
        batch=16,      # Batch size
        augment=True  # Enable data augmentation
    )
    results = model.val()
    print(results)

    return model

In [12]:
Hyena_model = train_pipeline(Hyena_dir)
Fishcat_model = train_pipeline(Fish_cat_dir)

Augmented folder: /content/drive/MyDrive/aug_Hyena
Processing category: H1


100%|██████████| 8/8 [00:05<00:00,  1.46it/s]


Processing category: H10


100%|██████████| 8/8 [00:04<00:00,  1.82it/s]


Processing category: H100


...

WARNING ⚠️ ClassificationModel does not support 'augment=True' prediction. Reverting to single-scale prediction.


               classes   top1_acc   top5_acc:  88%|████████▊ | 7/8 [00:17<00:02,  2.48s/it]

WARNING ⚠️ ClassificationModel does not support 'augment=True' prediction. Reverting to single-scale prediction.


               classes   top1_acc   top5_acc: 100%|██████████| 8/8 [00:18<00:00,  2.33s/it]


                   all      0.417      0.765
Speed: 0.0ms preprocess, 15.0ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train32
ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78fa07ba9a50>
curves: []
curves_results: []
fitness: 0.5913043320178986
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.417391300201416, 'metrics/accuracy_top5': 0.7652173638343811, 'fitness': 0.5913043320178986}
save_dir: PosixPath('runs/classify/train32')
speed: {'preprocess': 0.001967471578846807, 'inference': 15.023741514786431, 'loss': 0.00025707742442255437, 'postprocess': 0.00014097794242527175}
task: 'classify'
top1: 0.417391300201416
top5: 0.7652173638343811
